<a href="https://colab.research.google.com/github/jvrupp/Curso_UESC_Pandas_2022/blob/main/Diamantes2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Versao do execicio dos diamantes de forma mais simples e direta.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
diamonds= pd.read_csv('/content/gdrive/MyDrive/diamonds.csv')
##################################tratando as variaveis categoricas
cat_cut=['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
cat_color=['D','E','F','G','H','I','J']
cat_clarity='I1,SI2,SI1,VS2,VS1,VVS2,VVS1,IF'.split(',')
diamonds.drop(columns=['Unnamed: 0'],inplace=True)
diamonds['color']=diamonds['color'].replace(cat_color,[0,1,2,3,4,5,6])
diamonds['cut']=diamonds['cut'].replace(cat_cut,[0,1,2,3,4])
diamonds['clarity']=diamonds['clarity'].replace(cat_clarity,[0,1,2,3,4,5,6,7])

preco = diamonds['price']
diamonds.drop(columns=['price'],inplace=True)
diamonds2=diamonds.copy()#copia





In [7]:
for i in range(0,9):
    scaler=StandardScaler()
    diamonds.iloc[:,[i]]=scaler.fit_transform(diamonds.iloc[:,[i]])

In [8]:
kmeans=KMeans(n_clusters=5)
kmeans.fit(diamonds.iloc[:,[0,-1]])


diamonds2['grupos']=np.array(pd.Series(kmeans.labels_))
diamonds['grupos']=np.array(pd.Series(kmeans.labels_))

In [9]:
X_train,X_test,Y_train,Y_test=train_test_split(diamonds2,preco,test_size=0.10,random_state=0)

In [10]:
regr = RandomForestRegressor(n_estimators=100, random_state=0)
regr.fit(X_train, Y_train)
pred=regr.predict(X_test)
r2_score(Y_test,pred)

0.9823125513501234

In [11]:
diferencas = pd.DataFrame(abs(pred - Y_test))

diferencas['ones']=1

ImpReg=pd.DataFrame({'importanciaReg':regr.feature_importances_,'variaveis':X_train.columns})

In [12]:
###########################################
k3=KMeans(n_clusters=3)
k3.fit(diferencas)
classes_k3=k3.labels_
diferencas['gp_diff']=classes_k3

In [13]:
def JuntaRuins(qts,diferencas):
  means=[]
  min_index = qts.idxmin()
  Error_Data_Set = diferencas[diferencas['gp_diff']!=min_index]
  Not_Error_Data_Set = diferencas[diferencas['gp_diff']==min_index]
  return (Error_Data_Set,Not_Error_Data_Set)



In [14]:
bad_data,not_bad_data=JuntaRuins(diferencas.groupby(['gp_diff']).mean()['price'],diferencas)
bad_data=bad_data.rename(columns={'price':'diff'})
not_bad_data=not_bad_data.rename(columns={'price':'diff'})


Total_Data=pd.concat([bad_data, not_bad_data], axis=0) # 0 row 1 columns


Total_Data=X_test.merge(Total_Data,right_index=True,left_index=True,how='outer').drop(columns=['ones'])
Total_Data['gp_diff']=Total_Data['gp_diff'].replace([0,1,2],[0,1,1])

Total_Data['carat/vol']=Total_Data['x']*Total_Data['y']*Total_Data['z']
X_class_erros=Total_Data.iloc[:,0:-3]
X_class_erros['carat/vol']=Total_Data.iloc[:,-1]
Y_class_erros=Total_Data.iloc[:,-2]



X_train2,X_test2,Y_train2,Y_test2=train_test_split(X_class_erros,Y_class_erros,test_size=0.20,random_state=0)



regr = RandomForestClassifier(n_estimators=200, random_state=0)
regr.fit(X_train2, Y_train2)
pred2=regr.predict(X_test2)


print(classification_report(Y_test2,pred2))


              precision    recall  f1-score   support

           0       0.90      0.95      0.92       924
           1       0.56      0.37      0.45       155

    accuracy                           0.87      1079
   macro avg       0.73      0.66      0.69      1079
weighted avg       0.85      0.87      0.86      1079

